<a href="https://colab.research.google.com/github/Aisnya975/problemset_aisyah_033/blob/main/week10_soal678917.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import matplotlib.pyplot as plt

# ==============================================================================
# BAGIAN 1: DEFINISI FUNGSI (ALGORITMA)
# (Bagian ini adalah "mesin" penghitungnya, tidak perlu diubah)
# ==============================================================================

def solve_fcfs(head, requests, title="FCFS"):
    print(f"\n[{title}] Head Awal: {head}")
    current = head
    total_seek = 0
    path = [head]

    for req in requests:
        distance = abs(req - current)
        total_seek += distance
        # print(f"  Move {current} -> {req} (Jarak: {distance})") # Uncomment untuk detail per langkah
        current = req
        path.append(current)

    print(f"  > Urutan Akses: {path}")
    print(f"  > Total Head Movement: {total_seek}")
    return total_seek

def solve_sstf(head, requests, title="SSTF"):
    print(f"\n[{title}] Head Awal: {head}")
    current = head
    total_seek = 0
    path = [head]
    pending = requests.copy()

    while pending:
        closest_req = min(pending, key=lambda x: abs(x - current))
        distance = abs(closest_req - current)
        total_seek += distance
        current = closest_req
        path.append(current)
        pending.remove(current)

    print(f"  > Urutan Akses: {path}")
    print(f"  > Total Head Movement: {total_seek}")
    return total_seek

def solve_look(head, requests, direction='up', title="LOOK"):
    print(f"\n[{title}] Head Awal: {head}, Arah: {direction.upper()}")
    current = head
    total_seek = 0
    path = [head]

    left = sorted([r for r in requests if r < head])
    right = sorted([r for r in requests if r > head])

    # Logika LOOK: Tidak sampai ujung disk (0 atau 199), hanya sampai request terjauh
    if direction == 'up':
        run_queue = right + left[::-1] # Naik ke request tertinggi, lalu turun
    else:
        run_queue = left[::-1] + right # Turun ke request terendah, lalu naik

    for req in run_queue:
        total_seek += abs(req - current)
        current = req
        path.append(current)

    print(f"  > Urutan Akses: {path}")
    print(f"  > Total Head Movement: {total_seek}")
    return total_seek

def solve_cscan(head, requests, disk_max=199, title="C-SCAN"):
    print(f"\n[{title}] Head Awal: {head}, Limit Disk: {disk_max}")
    # Asumsi arah selalu UP (Increasing) untuk C-SCAN standar
    total_seek = 0
    path = [head]

    left = sorted([r for r in requests if r < head])
    right = sorted([r for r in requests if r > head])

    # 1. Fase Naik (ke kanan) sampai request terakhir di kanan
    current = head
    for req in right:
        path.append(req)
        current = req

    # 2. Hitung perjalanan ke ujung disk (199)
    # Note: Beberapa implementasi menghitung full seek sampai 199
    path.append(disk_max)

    # 3. Jump ke 0 (Return Sweep)
    path.append(0)

    # 4. Fase Naik lagi (dari 0 ke sisa request di kiri)
    for req in left:
        path.append(req)

    # Menghitung Total Movement
    # Rumus umum C-SCAN: (Dist ke Max) + (Jump ke 0) + (0 ke Max Request di kiri)
    # Jump biasanya dihitung sebagai jarak penuh (disk_max - 0) atau 0 tergantung dosen.
    # Di sini kita hitung jarak fisiknya:

    # a. Head ke 199
    seek_1 = abs(disk_max - head)
    # b. 199 ke 0 (Return sweep)
    seek_2 = disk_max
    # c. 0 ke request terakhir di kiri (max of left)
    seek_3 = max(left) if left else 0

    total_seek = seek_1 + seek_2 + seek_3

    print(f"  > Urutan Logis: {path}")
    print(f"  > Total Head Movement: {total_seek}")
    return total_seek

# ==============================================================================
# BAGIAN 2: JAWABAN PER NOMOR SOAL
# Data diambil langsung dari PDF Problem Set
# ==============================================================================

print("=== HASIL PERHITUNGAN PROBLEM SET MINGGU 10 ===")

# ------------------------------------------------------------------------------
# SOAL NOMOR 6 (Bagian C3)
# Data: Head 53, Queue: 98, 183, 37, 122, 14, 124, 65, 67
# Algoritma: FCFS
# ------------------------------------------------------------------------------
requests_soal_6 = [98, 183, 37, 122, 14, 124, 65, 67]
start_head_6 = 53
solve_fcfs(start_head_6, requests_soal_6, title="SOAL 6 - FCFS")


# ------------------------------------------------------------------------------
# SOAL NOMOR 7 (Bagian C3)
# Data: Sama dengan Soal 6
# Algoritma: SSTF
# ------------------------------------------------------------------------------
solve_sstf(start_head_6, requests_soal_6, title="SOAL 7 - SSTF")


# ------------------------------------------------------------------------------
# SOAL NOMOR 8 (Bagian C3)
# Data: Head 100, Queue: 55, 58, 39, 18, 90, 160, 150, 38, 184
# Arah: Kanan (Increasing/Up)
# Algoritma: LOOK
# ------------------------------------------------------------------------------
requests_soal_8 = [55, 58, 39, 18, 90, 160, 150, 38, 184]
start_head_8 = 100
solve_look(start_head_8, requests_soal_8, direction='up', title="SOAL 8 - LOOK")


# ------------------------------------------------------------------------------
# SOAL NOMOR 9 (Bagian C3)
# Data: Sama dengan Soal 8
# Algoritma: C-SCAN
# ------------------------------------------------------------------------------
solve_cscan(start_head_8, requests_soal_8, disk_max=199, title="SOAL 9 - C-SCAN")


# ------------------------------------------------------------------------------
# SOAL NOMOR 17 (Bagian C6) - Desain Algoritma
# Algoritma: Adaptive Weighted Look (AWL)
# Output: Pseudocode Design
# ------------------------------------------------------------------------------
print("\n[SOAL 17 - Desain Algoritma AWL]")
awl_pseudocode = """
ALGORITMA: Adaptive Weighted Look (AWL)
---------------------------------------
1. Setiap request memiliki atribut: Track, Priority (1-3), WaitingTime.
2. Loop Utama:
   a. Naikkan WaitingTime semua request di antrian.
   b. Cek Starvation: Jika WaitingTime > Threshold, set Priority = MAX (3).
   c. Filter arah: Ambil request yang searah dengan pergerakan Head.
   d. Sorting Kandidat:
      - Kunci 1 (Utama): Priority (High to Low)
      - Kunci 2 (Sekunder): Jarak terdekat (Shortest Seek)
   e. Jika tidak ada kandidat searah, balik arah (Reverse Direction).
   f. Eksekusi request terpilih.
"""
print(awl_pseudocode)


# ------------------------------------------------------------------------------
# SOAL NOMOR 19 (Bonus C6) - Konsep Next-Gen Scheduler
# Output: Klasifikasi Workload (Machine Learning Concept)
# ------------------------------------------------------------------------------
print("\n[SOAL 19 - BONUS: Konsep Workload Classifier]")
print("Logika Python untuk mendeteksi tipe workload:")
code_snippet_19 = """
def classify_workload(history_tracks):
    # Hitung varians (sebaran) data track
    variance = calculate_variance(history_tracks)

    if variance < 10:
        return "SEQUENTIAL (Video Streaming) -> Gunakan LOOK/FCFS"
    elif variance > 50:
        return "RANDOM (Web/Database) -> Gunakan C-LOOK/SSTF"
    else:
        return "MIXED -> Gunakan Adaptive Algorithm"
"""
print(code_snippet_19)

=== HASIL PERHITUNGAN PROBLEM SET MINGGU 10 ===

[SOAL 6 - FCFS] Head Awal: 53
  > Urutan Akses: [53, 98, 183, 37, 122, 14, 124, 65, 67]
  > Total Head Movement: 640

[SOAL 7 - SSTF] Head Awal: 53
  > Urutan Akses: [53, 65, 67, 37, 14, 98, 122, 124, 183]
  > Total Head Movement: 236

[SOAL 8 - LOOK] Head Awal: 100, Arah: UP
  > Urutan Akses: [100, 150, 160, 184, 90, 58, 55, 39, 38, 18]
  > Total Head Movement: 250

[SOAL 9 - C-SCAN] Head Awal: 100, Limit Disk: 199
  > Urutan Logis: [100, 150, 160, 184, 199, 0, 18, 38, 39, 55, 58, 90]
  > Total Head Movement: 388

[SOAL 17 - Desain Algoritma AWL]

ALGORITMA: Adaptive Weighted Look (AWL)
---------------------------------------
1. Setiap request memiliki atribut: Track, Priority (1-3), WaitingTime.
2. Loop Utama:
   a. Naikkan WaitingTime semua request di antrian.
   b. Cek Starvation: Jika WaitingTime > Threshold, set Priority = MAX (3).
   c. Filter arah: Ambil request yang searah dengan pergerakan Head.
   d. Sorting Kandidat:
      - 